In [1]:
import numpy as np

In [2]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [3]:
words, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

<img src="images/cosine_sim.png" style="width:800px;height:300px;">

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

In [4]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similarity between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    dot = np.dot(u, v)
    u_norm = np.sqrt(np.sum(u*u))
    v_norm = np.sqrt(np.sum(v*v))
    return dot/(u_norm*v_norm) 

In [5]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.2743924626137942
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174201


In [6]:

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    word_a = word_a.lower()
    word_b = word_b.lower()
    word_c = word_c.lower()
    a = word_to_vec_map[word_a]
    b = word_to_vec_map[word_b]
    c = word_to_vec_map[word_c]
    best_sim = -10
    best_word = 0
    for word in word_to_vec_map.keys():
        if word in [word_a, word_b, word_c]:
            continue
        current_sim = cosine_similarity(word_to_vec_map[word], b-a+c)
        if current_sim > best_sim:
            best_sim = current_sim
            best_word = word
    return best_word

In [7]:
print(complete_analogy("lion", "jungle", "human", word_to_vec_map))
print(complete_analogy("ghost", "fear", "love", word_to_vec_map))

isolated
desire


In [8]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger


In [9]:
# GENDER DIRECTION
g = word_to_vec_map["man"] - word_to_vec_map["woman"]
g

array([ 0.087144  , -0.2182    ,  0.40986   ,  0.03922   ,  0.1032    ,
       -0.94165   ,  0.06042   , -0.32988   , -0.46144   ,  0.35962   ,
       -0.31102   ,  0.86824   , -0.96006   , -0.01073   , -0.24337   ,
       -0.08193   ,  1.02722   ,  0.21122   , -0.695044  ,  0.00222   ,
       -0.29106   , -0.5053    ,  0.099454  , -0.40445   , -0.30181   ,
       -0.1355    ,  0.0606    ,  0.07131   ,  0.19245   ,  0.06115   ,
        0.3204    , -0.07165   ,  0.13337   ,  0.25068714,  0.14293   ,
        0.224957  ,  0.149     , -0.048882  , -0.12191   ,  0.27362   ,
        0.165476  ,  0.20426   , -0.54376   ,  0.271425  ,  0.10245   ,
        0.32108   , -0.2516    ,  0.33455   ,  0.04371   , -0.01258   ])

In [10]:
name_list = ["rahul", "priya", "god", "goddess", "king", "queen"]
for name in name_list:
    print(f"{name} {cosine_similarity(word_to_vec_map[name], g)}")
# MALE = +ve
# FEMALE = -ve

rahul 0.16915471039231722
priya -0.17632041839009402
god 0.20540881117974222
goddess -0.1406848269239382
king 0.18875684078397284
queen -0.2044893354882803


In [11]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick -0.27691916256382665
guns 0.1888485567898898
science 0.06082906540929699
arts -0.008189312385880344
literature -0.0647250443345993
warrior 0.20920164641125288
doctor -0.11895289410935045
tree 0.07089399175478092
receptionist -0.3307794175059374
technology 0.13193732447554293
fashion -0.035638946257727
teacher -0.1792092343182567
engineer 0.08039280494524072
pilot -0.0010764498991917074
computer 0.10330358873850498
singer -0.18500518136496297


Do you notice anything surprising? It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, "computer" is closer to "man" while "literature" is closer to "woman". Ouch! 

We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You will have to treat these two types of words differently when debiasing.

<img src="images/neutral.png" style="width:800px;height:300px;">

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

In [12]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    e = word_to_vec_map[word.lower()]
    e_bias = (np.dot(e, g)/np.sum(g*g))*g
    e_debiased = e - e_bias
    return e_debiased

In [13]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  -0.3307794175059374
cosine similarity between receptionist and g, after neutralizing:  4.442232511624783e-17


### Equalization algorithm for gender-specific words

Next, lets see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. In pictures, this is how equalization works: 

<img src="images/equalize10.png" style="width:800px;height:400px;">


The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) 

In [14]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    w1, w2 = pair
    w1 = w1.lower()
    w2 = w2.lower()
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]

    mu = (e_w1+e_w2)/2

    mu_B = (np.dot(mu, bias_axis)/np.sum(bias_axis*bias_axis)) *bias_axis
    mu_orth = mu - mu_B
     # Step 4: Use equations (7) and (8) to compute e_w1B and e_w2B (≈2 lines)
    e_w1B = np.dot(e_w1, bias_axis) / np.sum(bias_axis * bias_axis) * bias_axis
    e_w2B = np.dot(e_w2, bias_axis) / np.sum(bias_axis * bias_axis) * bias_axis
        
    # Step 5: Adjust the Bias part of e_w1B and e_w2B using the formulas (9) and (10) given above (≈2 lines)
    corrected_e_w1B = np.sqrt(np.abs(1 - np.sum(mu_orth * mu_orth))) * (e_w1B - mu_B) / np.linalg.norm(e_w1 - mu_orth - mu_B)
    corrected_e_w2B = np.sqrt(np.abs(1 - np.sum(mu_orth * mu_orth))) * (e_w2B - mu_B) / np.linalg.norm(e_w2 - mu_orth - mu_B)

    # Step 6: Debias by equalizing e1 and e2 to the sum of their corrected projections (≈2 lines)
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
                                                                
    ### END CODE HERE ###
    
    return e1, e2 

In [15]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  0.11711095765336832
cosine_similarity(word_to_vec_map["woman"], gender) =  -0.35666618846270376

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  0.7004364289309388
cosine_similarity(e2, gender) =  -0.7004364289309388
